In [83]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
import pickle
import os
import numpy as np
import tensorflow as tf
import keras.backend as K
import matplotlib.pyplot as plt
from sklearn import model_selection
from tqdm.notebook import tqdm
import umap
import time

BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")
import utils.papers

In [85]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
RANDOM_SEED = CONFIG['random_seed']
# type of noise
# asym: classes flip to a single other class
# sym: classes flip uniformly to any other class
TYPE = CONFIG["experiment_configs"][DATASET]["type"]
 # chance of flip
NOISE_P = CONFIG["experiment_configs"][DATASET]["noise_p"]
HYPER_VAL_SPLIT = CONFIG["experiment_configs"][DATASET]["hyper_val_split"]

EPOCHS = CONFIG["experiment_configs"][DATASET]["epochs"]
BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]
IMAGE_X = CONFIG["experiment_configs"][DATASET]["image_x_size"]
IMAGE_Y = CONFIG["experiment_configs"][DATASET]["image_y_size"]
IMAGE_SIZE = (IMAGE_Y, IMAGE_X)

print(RANDOM_SEED, TYPE, NOISE_P)

# folders for processed, models
PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/{DATASET}/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/{DATASET}/rs={RANDOM_SEED}')

PROCESSED_SAVEPATH = utils.utils.get_savepath(PROCESSED_DIR, DATASET, ".npz", t=TYPE, np=NOISE_P)
BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="base", t=TYPE, np=NOISE_P)

if not os.path.exists(BASE_MODEL_SAVEPATH):
    print(f"warning: model has not been run for rs={RANDOM_SEED}_t={TYPE}_np={NOISE_P}")

55 asym 0.6


In [86]:
with open(PROCESSED_SAVEPATH, 'rb') as f:
    dat = np.load(f)

    x_train = dat['x_train']
    y_train = dat['y_train']

    x_hyper_train = dat['x_hyper_train']
    y_hyper_train = dat['y_hyper_train']
    
    x_val = dat['x_val']
    y_val = dat['y_val']
    
    x_hyper_val = dat['x_hyper_val']
    y_hyper_val = dat['y_hyper_val']

    x_test = dat['x_test']
    y_test = dat['y_test']

x_train_full = np.concatenate([x_train, x_hyper_train])
y_train_full = np.concatenate([y_train, y_hyper_train])

x_val_full = np.concatenate([x_val, x_hyper_val])
y_val_full = np.concatenate([y_val, y_hyper_val])

In [87]:
y_train = tf.keras.utils.to_categorical(y_train)
y_hyper_train = tf.keras.utils.to_categorical(y_hyper_train)
y_train_full = tf.keras.utils.to_categorical(y_train_full)

y_val = tf.keras.utils.to_categorical(y_val)
y_hyper_val = tf.keras.utils.to_categorical(y_hyper_val)
y_val_full = tf.keras.utils.to_categorical(y_val_full)

y_test = tf.keras.utils.to_categorical(y_test)

In [88]:
model = utils.utils.make_resnet(
    2,
    random_state=RANDOM_SEED,
    input_shape=(*IMAGE_SIZE, 3),
    nc=10,
)
model.load_weights(BASE_MODEL_SAVEPATH)

In [7]:
# train acc
preds_train = utils.utils.compute_preds(
    model,
    x_train,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_train, axis=1) == np.argwhere(y_train)[:,1]).mean()

0.8012894736842106

In [8]:
# val acc
preds_val = utils.utils.compute_preds(
    model,
    x_val_full,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_val, axis=1) == np.argwhere(y_val_full)[:,1]).mean()

0.5815

In [9]:
# test acc
preds_test = utils.utils.compute_preds(
    model,
    x_test,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_test, axis=1) == np.argwhere(y_test)[:,1]).mean()

0.5772

# Baseline 1: Fine-Tune
This is a very widely used technique in deep learning. The idea is simple: do a little bit more training on the validation set.

In [89]:
# load the base weights
model.load_weights(BASE_MODEL_SAVEPATH)

In [90]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

In [91]:
optimizer = tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9, decay=0.0)

In [92]:
model.compile(optimizer, loss='categorical_crossentropy', metrics = ['accuracy'])

In [93]:
FT_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="ft", t=TYPE, np=NOISE_P)

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=FT_MODEL_SAVEPATH,
    monitor="val_loss",
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

def scheduler(epoch):
    if epoch > 80:
        return 1e-6
    elif epoch > 40:
        return 1e-5
    else:
        return 1e-4

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# callbacks = [lr_scheduler, save_best]
callbacks = [save_best]

In [95]:
model.fit(
    x=data_generator.flow(
        x_val,
        y_val,
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=RANDOM_SEED,
    ),
    validation_data=(x_hyper_val, y_hyper_val),
    epochs=EPOCHS,
    callbacks=callbacks,
)

Epoch 1/120
4/4 [==============================] - ETA: 0s - loss: 0.9461 - accuracy: 0.5380
Epoch 00001: val_loss improved from inf to 0.86047, saving model to ../../../models/cifar10/rs=55/cifar10_mt=ft_np=0.6_t=asym.h5
4/4 [==============================] - 1s 293ms/step - loss: 0.9461 - accuracy: 0.5380 - val_loss: 0.8605 - val_accuracy: 0.5940
Epoch 2/120
4/4 [==============================] - ETA: 0s - loss: 0.9350 - accuracy: 0.5500
Epoch 00002: val_loss improved from 0.86047 to 0.85845, saving model to ../../../models/cifar10/rs=55/cifar10_mt=ft_np=0.6_t=asym.h5
4/4 [==============================] - 0s 97ms/step - loss: 0.9350 - accuracy: 0.5500 - val_loss: 0.8585 - val_accuracy: 0.5940
Epoch 3/120
4/4 [==============================] - ETA: 0s - loss: 0.9170 - accuracy: 0.5440
Epoch 00003: val_loss improved from 0.85845 to 0.85625, saving model to ../../../models/cifar10/rs=55/cifar10_mt=ft_np=0.6_t=asym.h5
4/4 [==============================] - 0s 110ms/step - loss: 0.9170 -

4/4 [==============================] - ETA: 0s - loss: 0.8058 - accuracy: 0.6460
Epoch 00047: val_loss improved from 0.75963 to 0.75797, saving model to ../../../models/cifar10/rs=55/cifar10_mt=ft_np=0.6_t=asym.h5
4/4 [==============================] - 0s 96ms/step - loss: 0.8058 - accuracy: 0.6460 - val_loss: 0.7580 - val_accuracy: 0.7000
Epoch 48/120
4/4 [==============================] - ETA: 0s - loss: 0.7372 - accuracy: 0.6820
Epoch 00048: val_loss improved from 0.75797 to 0.75647, saving model to ../../../models/cifar10/rs=55/cifar10_mt=ft_np=0.6_t=asym.h5
4/4 [==============================] - 0s 96ms/step - loss: 0.7372 - accuracy: 0.6820 - val_loss: 0.7565 - val_accuracy: 0.7000
Epoch 49/120
4/4 [==============================] - ETA: 0s - loss: 0.7654 - accuracy: 0.6880
Epoch 00049: val_loss improved from 0.75647 to 0.75491, saving model to ../../../models/cifar10/rs=55/cifar10_mt=ft_np=0.6_t=asym.h5
4/4 [==============================] - 0s 96ms/step - loss: 0.7654 - accurac

4/4 [==============================] - ETA: 0s - loss: 0.6265 - accuracy: 0.7580
Epoch 00093: val_loss improved from 0.70977 to 0.70908, saving model to ../../../models/cifar10/rs=55/cifar10_mt=ft_np=0.6_t=asym.h5
4/4 [==============================] - 0s 93ms/step - loss: 0.6265 - accuracy: 0.7580 - val_loss: 0.7091 - val_accuracy: 0.7560
Epoch 94/120
4/4 [==============================] - ETA: 0s - loss: 0.6681 - accuracy: 0.7480
Epoch 00094: val_loss improved from 0.70908 to 0.70832, saving model to ../../../models/cifar10/rs=55/cifar10_mt=ft_np=0.6_t=asym.h5
4/4 [==============================] - 0s 95ms/step - loss: 0.6681 - accuracy: 0.7480 - val_loss: 0.7083 - val_accuracy: 0.7560
Epoch 95/120
4/4 [==============================] - ETA: 0s - loss: 0.6276 - accuracy: 0.7680
Epoch 00095: val_loss improved from 0.70832 to 0.70746, saving model to ../../../models/cifar10/rs=55/cifar10_mt=ft_np=0.6_t=asym.h5
4/4 [==============================] - 0s 95ms/step - loss: 0.6276 - accurac

In [96]:
# see the best saved
model.load_weights(FT_MODEL_SAVEPATH)

In [97]:
# val acc
preds_val = utils.utils.compute_preds(
    model,
    x_hyper_val,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_val, axis=1) == np.argwhere(y_hyper_val)[:,1]).mean()

0.778

In [98]:
# test acc
preds_test = utils.utils.compute_preds(
    model,
    x_test,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_test, axis=1) == np.argwhere(y_test)[:,1]).mean()

0.7529

# Baseline 2: Forward Correct

Paper: https://arxiv.org/pdf/1609.03683.pdf

In [19]:
# reload trained weights
model.load_weights(BASE_MODEL_SAVEPATH)

In [21]:
# T[i,j] means the probabilty that class i is flipped to class j.
T_hat = utils.papers.forward_correct_est_T(
    model,
    x_train_full,
    nc=10,
    batch_size=BATCH_SIZE,
)

In [22]:
np.round(T_hat, decimals=2)

array([[1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.6 , 0.  , 0.4 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.59, 0.  , 0.41, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.01, 0.41, 0.01, 0.  , 0.58, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.35, 0.  , 0.65, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.53, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.47]])

In [23]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

In [24]:
optimizer = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9)

In [25]:
loss = utils.papers.make_forward_correct_loss(T_hat)

In [26]:
model.compile(optimizer, loss=loss, metrics = ['accuracy'])

In [27]:
FC_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="fc", t=TYPE, np=NOISE_P)

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=FC_MODEL_SAVEPATH,
    monitor="val_loss",
    mode='max',
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
    verbose=1,
    mode="auto",
)

callbacks = [save_best, early_stop]

In [28]:
# there are other ways to do this, namely
# fit to x_train, y_train and see what minimizes the loss on x_hyper_train
# because the paper says to minimize the modified loss
history = model.fit(
    x=data_generator.flow(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=RANDOM_SEED,
    ),
    epochs=EPOCHS,
    verbose=1,
    validation_data=(x_hyper_train, y_hyper_train),
    callbacks=callbacks,
)

Epoch 1/120
387/387 [==============================] - ETA: 0s - loss: 0.5450 - accuracy: 0.7575
Epoch 00001: val_accuracy improved from -inf to 0.75400, saving model to ../../../models/cifar10/rs=15/cifar10_mt=fc_np=0.6_t=asym.h5
387/387 [==============================] - 19s 50ms/step - loss: 0.5450 - accuracy: 0.7575 - val_loss: 0.8389 - val_accuracy: 0.7540
Epoch 2/120
387/387 [==============================] - ETA: 0s - loss: 0.5373 - accuracy: 0.7331
Epoch 00002: val_accuracy improved from 0.75400 to 0.76200, saving model to ../../../models/cifar10/rs=15/cifar10_mt=fc_np=0.6_t=asym.h5
387/387 [==============================] - 19s 50ms/step - loss: 0.5373 - accuracy: 0.7331 - val_loss: 0.8085 - val_accuracy: 0.7620
Epoch 3/120
387/387 [==============================] - ETA: 0s - loss: 0.5314 - accuracy: 0.7329
Epoch 00003: val_accuracy improved from 0.76200 to 0.76400, saving model to ../../../models/cifar10/rs=15/cifar10_mt=fc_np=0.6_t=asym.h5
387/387 [==========================

In [29]:
model.load_weights(FC_MODEL_SAVEPATH)

In [30]:
# val acc
preds_val = utils.utils.compute_preds(
    model,
    x_val_full,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_val, axis=1) == np.argwhere(y_val_full)[:,1]).mean()

0.764

In [31]:
# test acc
preds_test = utils.utils.compute_preds(
    model,
    x_test,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_test, axis=1) == np.argwhere(y_test)[:,1]).mean()

0.7711

# Baseline 3: Learn to Weigh Examples
Paper: https://arxiv.org/pdf/1803.09050.pdf

This is a type of meta-learning, which doesn't quite work with the keras API. We will need to manually implement the training loop.

In [20]:
model.load_weights(BASE_MODEL_SAVEPATH)

In [21]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

In [22]:
optimizer = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9, decay=0.0)

In [23]:
# iterator to generate data
itr = data_generator.flow(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=RANDOM_SEED,
)

In [24]:
# Reduction.NONE means the cross entropy is computed per entry in the batch
# but is not aggregated. Traditional cross entropy will average the results.
ce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

In [25]:
best_loss = float('inf')
LRW_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="lrw", t=TYPE, np=NOISE_P)

total_batches = len(itr)

# custom train loop
for epoch in range(EPOCHS):
    # implements a train loop
    print(f"\nEpoch {epoch}:\n----------")
    
    batch_c = 0
    loss_sum = 0
    for batch, labels in tqdm(itr):
        # most of the details are abstracted away in this function
        loss = utils.papers.train_step(model, batch, labels, x_val, y_val, ce, optimizer)
        loss_sum += loss
        batch_c += 1
        # print ongoing avg loss
        print(f"Loss: {loss_sum / batch_c}", end='\r')
        
        # we need to tally this ourselves because the iterator simply restarts another epoch
        if batch_c >= total_batches:
            break
    
    # compute validation accuracy
    preds = utils.utils.compute_preds(
        model,
        x_hyper_val,
        batch_size=BATCH_SIZE,
    )
    val_acc = (np.argmax(preds, axis=1) == np.argwhere(y_hyper_val)[:,1]).mean()
    loss_avg = loss_sum / total_batches
    
    print(f"Hyper Val Acc: {val_acc}")
    print(f"Hyper Val Loss: {loss_avg}")
    
    # implements save best logic
    if loss_avg < best_loss:
        best_loss = loss_avg
        print(f"Saving new best weights to {LRW_MODEL_SAVEPATH}")
        model.save_weights(
            filepath=LRW_MODEL_SAVEPATH,
            save_format="h5",
        )
        last_best_epoch = epoch
    
    # because this takes a long time (roughly 3x the normal train time)
    # we use early stopping 
    early_stop = 10
    if last_best_epoch + early_stop <= epoch:
        print(f"no improvement for {early_stop} epochs, ending training")
        break
        

Epoch 0:
----------


Loss: 0.190139353275299075
Val Test Acc: 0.544
Val Test Loss: 0.19013935327529907

Saving new best weights to ../../../models/cifar10/rs=15/cifar10_mt=lrw_np=0.6_t=asym.h5
Epoch 1:
----------


Loss: 0.16814349591732025
Val Test Acc: 0.576
Val Test Loss: 0.16814349591732025

Saving new best weights to ../../../models/cifar10/rs=15/cifar10_mt=lrw_np=0.6_t=asym.h5
Epoch 2:
----------


Loss: 0.145715013146400454
Val Test Acc: 0.556
Val Test Loss: 0.14571501314640045

Saving new best weights to ../../../models/cifar10/rs=15/cifar10_mt=lrw_np=0.6_t=asym.h5
Epoch 3:
----------


Loss: 0.17932464182376862
Val Test Acc: 0.568
Val Test Loss: 0.17932464182376862

Epoch 4:
----------


...

Loss: 0.18691381812095642
Val Test Acc: 0.584
Val Test Loss: 0.18691381812095642

Epoch 12:
----------


Loss: 0.19280989468097687
Val Test Acc: 0.58
Val Test Loss: 0.19280989468097687

Epoch 13:
----------


Loss: 0.18195927143096924
Val Test Acc: 0.58
Val Test Loss: 0.18195927143096924

Epoch 14:
----------


Loss: 0.16227981448173523
Val Test Acc: 0.592
Val Test Loss: 0.16227981448173523

Epoch 15:
----------


Loss: 0.22039318084716797
Val Test Acc: 0.564
Val Test Loss: 0.22039318084716797

no improvement for 10 epochs, ending training


In [26]:
model.load_weights(LRW_MODEL_SAVEPATH)

In [27]:
# val acc
preds_val = utils.utils.compute_preds(
    model,
    x_hyper_val,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_val, axis=1) == np.argwhere(y_hyper_val)[:,1]).mean()

0.58

In [28]:
# test acc
preds_test = utils.utils.compute_preds(
    model,
    x_test,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_test, axis=1) == np.argwhere(y_test)[:,1]).mean()

0.6336

# Baseline 4: Just Train on Validation Set

In [53]:
# make a fresh model
model = utils.utils.make_resnet(
    depth=2,
    random_state=RANDOM_SEED,
    input_shape=(*IMAGE_SIZE, 3),
    nc=10,
)

In [54]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_30 (Conv2D)              (None, 32, 32, 16)   448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_26 (BatchNo (None, 32, 32, 16)   64          conv2d_30[0][0]                  
__________________________________________________________________________________________________
activation_26 (Activation)      (None, 32, 32, 16)   0           batch_normalization_26[0][0]     
_______________________________________________________________________________________

In [55]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

In [56]:
optimizer = tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=0.0)

In [57]:
model.compile(optimizer, loss='categorical_crossentropy', metrics = ['accuracy'])

In [58]:
def scheduler(epoch):
    if epoch > 80:
        return 0.001
    elif epoch > 40:
        return 0.01
    else:
        return 0.1

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [59]:
JV_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="jv", t=TYPE, np=NOISE_P)
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=JV_MODEL_SAVEPATH,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
)

In [60]:
callbacks = [lr_scheduler, save_best]

In [61]:
history = model.fit(
        x=data_generator.flow(
            x_val,
            y_val,
            batch_size=BATCH_SIZE,
            shuffle=True,
            seed=RANDOM_SEED,
        ),
        epochs=EPOCHS,
        validation_data=(x_hyper_val, y_hyper_val),
        verbose=1,
        callbacks=callbacks,
)

Epoch 1/120
2/2 [==============================] - ETA: 0s - loss: 2.5918 - accuracy: 0.1080
Epoch 00001: val_accuracy improved from -inf to 0.10000, saving model to ../../../models/cifar10/rs=15/cifar10_mt=jv_np=0.6_t=asym.h5
2/2 [==============================] - 1s 291ms/step - loss: 2.5918 - accuracy: 0.1080 - val_loss: 56.5558 - val_accuracy: 0.1000
Epoch 2/120
1/2 [==============>...............] - ETA: 0s - loss: 2.4779 - accuracy: 0.1328
Epoch 00002: val_accuracy improved from 0.10000 to 0.10400, saving model to ../../../models/cifar10/rs=15/cifar10_mt=jv_np=0.6_t=asym.h5
2/2 [==============================] - 0s 85ms/step - loss: 2.3948 - accuracy: 0.1760 - val_loss: 33.3243 - val_accuracy: 0.1040
Epoch 3/120
2/2 [==============================] - ETA: 0s - loss: 2.2398 - accuracy: 0.2320
Epoch 00003: val_accuracy improved from 0.10400 to 0.11200, saving model to ../../../models/cifar10/rs=15/cifar10_mt=jv_np=0.6_t=asym.h5
2/2 [==============================] - 0s 90ms/step - 

Epoch 29/120
1/2 [==============>...............] - ETA: 0s - loss: 1.3041 - accuracy: 0.5820
Epoch 00029: val_accuracy did not improve from 0.20000
2/2 [==============================] - 0s 47ms/step - loss: 1.3517 - accuracy: 0.5600 - val_loss: 21.2465 - val_accuracy: 0.1320
Epoch 30/120
1/2 [==============>...............] - ETA: 0s - loss: 1.3078 - accuracy: 0.5984
Epoch 00030: val_accuracy did not improve from 0.20000
2/2 [==============================] - 0s 47ms/step - loss: 1.2942 - accuracy: 0.6240 - val_loss: 19.1975 - val_accuracy: 0.1360
Epoch 31/120
2/2 [==============================] - ETA: 0s - loss: 1.2598 - accuracy: 0.6200
Epoch 00031: val_accuracy did not improve from 0.20000
2/2 [==============================] - 0s 51ms/step - loss: 1.2598 - accuracy: 0.6200 - val_loss: 21.8771 - val_accuracy: 0.1400
Epoch 32/120
1/2 [==============>...............] - ETA: 0s - loss: 1.2255 - accuracy: 0.6719
Epoch 00032: val_accuracy did not improve from 0.20000
2/2 [============

2/2 [==============================] - 0s 84ms/step - loss: 0.9096 - accuracy: 0.7440 - val_loss: 8.9411 - val_accuracy: 0.2160
Epoch 59/120
1/2 [==============>...............] - ETA: 0s - loss: 0.8064 - accuracy: 0.8125
Epoch 00059: val_accuracy improved from 0.21600 to 0.22400, saving model to ../../../models/cifar10/rs=15/cifar10_mt=jv_np=0.6_t=asym.h5
2/2 [==============================] - 0s 86ms/step - loss: 0.8800 - accuracy: 0.7560 - val_loss: 8.3250 - val_accuracy: 0.2240
Epoch 60/120
2/2 [==============================] - ETA: 0s - loss: 0.8422 - accuracy: 0.7720
Epoch 00060: val_accuracy improved from 0.22400 to 0.23600, saving model to ../../../models/cifar10/rs=15/cifar10_mt=jv_np=0.6_t=asym.h5
2/2 [==============================] - 0s 87ms/step - loss: 0.8422 - accuracy: 0.7720 - val_loss: 7.7049 - val_accuracy: 0.2360
Epoch 61/120
1/2 [==============>...............] - ETA: 0s - loss: 0.8498 - accuracy: 0.7891
Epoch 00061: val_accuracy improved from 0.23600 to 0.24000, 

Epoch 114/120
2/2 [==============================] - ETA: 0s - loss: 0.4838 - accuracy: 0.9200
Epoch 00114: val_accuracy did not improve from 0.34400
2/2 [==============================] - 0s 53ms/step - loss: 0.4838 - accuracy: 0.9200 - val_loss: 2.6914 - val_accuracy: 0.3400
Epoch 115/120
2/2 [==============================] - ETA: 0s - loss: 0.4840 - accuracy: 0.9240
Epoch 00115: val_accuracy did not improve from 0.34400
2/2 [==============================] - 0s 51ms/step - loss: 0.4840 - accuracy: 0.9240 - val_loss: 2.6857 - val_accuracy: 0.3440
Epoch 116/120
1/2 [==============>...............] - ETA: 0s - loss: 0.4896 - accuracy: 0.9219
Epoch 00116: val_accuracy did not improve from 0.34400
2/2 [==============================] - 0s 48ms/step - loss: 0.4708 - accuracy: 0.9280 - val_loss: 2.6815 - val_accuracy: 0.3400
Epoch 117/120
1/2 [==============>...............] - ETA: 0s - loss: 0.5030 - accuracy: 0.8852
Epoch 00117: val_accuracy did not improve from 0.34400
2/2 [===========

In [62]:
model.load_weights(JV_MODEL_SAVEPATH)

In [63]:
# val acc
preds_val = utils.utils.compute_preds(
    model,
    x_hyper_val,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_val, axis=1) == np.argwhere(y_hyper_val)[:,1]).mean()

0.558

In [64]:
# test acc
preds_test = utils.utils.compute_preds(
    model,
    x_test,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds_test, axis=1) == np.argwhere(y_test)[:,1]).mean()

0.3269